In [20]:
import openai, json
import urllib.request
import urllib.error

client = openai.OpenAI()
messages = []
api_url = "https://nomad-movies.nomadcoders.workers.dev"

In [21]:
def fetch_api(path):
    url = f"{api_url}{path}"
    request = urllib.request.Request(
        url,
        headers={
            "User-Agent": "Mozilla/5.0",
            "Accept": "application/json",
        },
    )

    try:
        with urllib.request.urlopen(request, timeout=10) as response:
            return response.read().decode("utf-8")
    except urllib.error.HTTPError as e:
        return json.dumps({"error": f"HTTPError {e.code}: {e.reason}", "url": url})
    except urllib.error.URLError as e:
        return json.dumps({"error": f"URLError: {e.reason}", "url": url})


def get_popular_movies():
    return fetch_api("/movies")


def get_movie_details(id):
    return fetch_api(f"/movies/{id}")


def get_similar_movies(id):
    return fetch_api(f"/movies/{id}/similar")


FUNCTION_MAP = {
    "get_popular_movies": get_popular_movies,
    "get_movie_details": get_movie_details,
    "get_similar_movies": get_similar_movies,
}

In [22]:
TOOLS = [
    {
        "type": "function",
        "function": {
            "name": "get_popular_movies",
            "description": "A function to get the most popular movies.",
            "parameters": {
                "type": "object",
                "properties": {},
                "required": [],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "get_movie_details",
            "description": "A function to get the details of a movie.",
            "parameters": {
                "type": "object",
                "properties": {
                    "id": {
                        "type": "string",
                        "description": "The id of the movie to get the details of.",
                    }
                },
                "required": ["id"],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "get_similar_movies",
            "description": "A function to get the similar movies of a movie.",
            "parameters": {
                "type": "object",
                "properties": {
                    "id": {
                        "type": "string",
                        "description": "The id of the movie to get the similar movies of.",
                    }
                },
                "required": ["id"],
            },
        },
    }
    
]

In [23]:
from openai.types.chat import ChatCompletionMessage


def process_ai_response(message: ChatCompletionMessage):

    if message.tool_calls:
        messages.append(
            {
                "role": "assistant",
                "content": message.content or "",
                "tool_calls": [
                    {
                        "id": tool_call.id,
                        "type": "function",
                        "function": {
                            "name": tool_call.function.name,
                            "arguments": tool_call.function.arguments,
                        },
                    }
                    for tool_call in message.tool_calls
                ],
            }
        )

        for tool_call in message.tool_calls:
            function_name = tool_call.function.name
            arguments = tool_call.function.arguments

            print(f"Calling function: {function_name} with {arguments}")

            try:
                arguments = json.loads(arguments)
            except json.JSONDecodeError:
                arguments = {}

            function_to_run = FUNCTION_MAP.get(function_name)
            if not function_to_run:
                result = json.dumps({"error": f"Unknown function: {function_name}"})
            else:
                try:
                    result = function_to_run(**arguments)
                except Exception as e:
                    result = json.dumps(
                        {
                            "error": f"Function execution failed: {str(e)}",
                            "function": function_name,
                            "arguments": arguments,
                        }
                    )

            print(f"Ran {function_name} with args {arguments} for a result of {result}")

            messages.append(
                {
                    "role": "tool",
                    "tool_call_id": tool_call.id,
                    "name": function_name,
                    "content": result,
                }
            )

        call_ai()
    else:
        messages.append({"role": "assistant", "content": message.content})
        print(f"AI: {message.content}")


def call_ai():
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=messages,
        tools=TOOLS,
    )
    process_ai_response(response.choices[0].message)

In [24]:
while True:
    message = input("Send a message to the LLM...")
    if message == "quit" or message == "q":
        break
    else:
        messages.append({"role": "user", "content": message})
        print(f"User: {message}")
        call_ai()

User: tell me popular movies
Calling function: get_popular_movies with {}
Ran get_popular_movies with args {} for a result of [{"adult":false,"backdrop_path":"https://image.tmdb.org/t/p/w1280/6YjnTRBz704LF1uJ3ZC4wsS9T8r.jpg","genre_ids":[28,80,53],"id":1290821,"original_language":"en","original_title":"Shelter","overview":"A man living in self-imposed exile on a remote island rescues a young girl from a violent storm, setting off a chain of events that forces him out of seclusion to protect her from enemies tied to his past.","popularity":456.5931,"poster_path":"https://image.tmdb.org/t/p/w780/buPFnHZ3xQy6vZEHxbHgL1Pc6CR.jpg","release_date":"2026-01-28","title":"Shelter","video":false,"vote_average":6.9,"vote_count":140},{"adult":false,"backdrop_path":"https://image.tmdb.org/t/p/w1280/hYgUkH7TusddHRtelj53I6gFOWR.jpg","genre_ids":[28,53],"id":799882,"original_language":"en","original_title":"The Bluff","overview":"When her tranquil life on a remote island is shattered by the return of h

In [25]:
message

'q'